# Data in rows

In [16]:
import quasardb
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [17]:
sample_size=10
print(quasardb.version()) 
c = quasardb.Cluster("qdb://qdb-server:2836")

3.4.1


<b>Create table with two columns with different types</b>

In [18]:
#create table for this sample
table = c.ts("ts_batch1_1")

columns = [quasardb.ColumnInfo(quasardb.ColumnType.Double, "close"),
           quasardb.ColumnInfo(quasardb.ColumnType.Int64, "volume")]
table.create(columns)


<b> Create a transaction holder: a batch <br> Batch size is fixed programatically. Here we use the sample_size we have set </b>

In [19]:
batch_columns = [quasardb.BatchColumnInfo("ts_batch1_1", "close", sample_size),
                 quasardb.BatchColumnInfo("ts_batch1_1", "volume", sample_size)]
batch_inserter = c.ts_batch(batch_columns)

<b>Create data to be added to batch transaction</b>

In [20]:
#create timestamps to associate to timeseries
timestamps = np.array('2019-03-14', dtype=np.datetime64)+ np.arange(sample_size)

# Generate random doubles of timeserie
double_values = np.random.uniform(1, 100.0, sample_size)
# Generate random doubles of timeserie
int_values = np.random.randint(1, 100, sample_size)


<b>Add rows to batch transaction</b>

In [21]:
for row in range(len(timestamps)):
    batch_inserter.start_row(np.datetime64(timestamps[row], 'ns'))
    batch_inserter.set_double(0, double_values[row]) # set close
    batch_inserter.set_int64(1, int_values[row]) # set volume

<b> let's see if these entries made it to the server</b>

In [22]:
q = c.query("select * from ts_batch1_1 ")
res = q.run()
print ( "Table returned:", len( res.tables))


Table returned: 0


<b>The data will be pushed to the server as a single transaction, guaranteeing all data is committed in one operation</b>

In [23]:
# send to the server
batch_inserter.push()

<b>The data is now queryable</b>

In [24]:
q = c.query("select * from ts_batch1_1 ")
res = q.run()

for col in res.tables["ts_batch1_1"]:
    # col.name is a string for the name of the column
    # col.data is a numpy array of the proper type
    assert len(col.data) ==sample_size
    print(col.name, ": ", col.data)


$timestamp :  ['2019-03-14T00:00:00.000000000' '2019-03-15T00:00:00.000000000'
 '2019-03-16T00:00:00.000000000' '2019-03-17T00:00:00.000000000'
 '2019-03-18T00:00:00.000000000' '2019-03-19T00:00:00.000000000'
 '2019-03-20T00:00:00.000000000' '2019-03-21T00:00:00.000000000'
 '2019-03-22T00:00:00.000000000' '2019-03-23T00:00:00.000000000']
close :  [75.68424133 52.2474901  92.13038454 59.42907505 57.19005246 16.47828926
 50.99041598 75.97125142 97.64092279 45.21851715]
volume :  [20 18 37 60 20 88  4 29 79 12]


<b>You can specify ranges </b>

In [25]:
q = c.query("select * from ts_batch1_1 in range(now(), -20y)")
res = q.run()

for col in res.tables["ts_batch1_1"]:
    assert len(col.data) ==sample_size
    print(col.name, ": ", col.data)



$timestamp :  ['2019-03-14T00:00:00.000000000' '2019-03-15T00:00:00.000000000'
 '2019-03-16T00:00:00.000000000' '2019-03-17T00:00:00.000000000'
 '2019-03-18T00:00:00.000000000' '2019-03-19T00:00:00.000000000'
 '2019-03-20T00:00:00.000000000' '2019-03-21T00:00:00.000000000'
 '2019-03-22T00:00:00.000000000' '2019-03-23T00:00:00.000000000']
close :  [75.68424133 52.2474901  92.13038454 59.42907505 57.19005246 16.47828926
 50.99041598 75.97125142 97.64092279 45.21851715]
volume :  [20 18 37 60 20 88  4 29 79 12]


<b>Operations</b>

In [26]:
q = c.query("select min(close) from ts_batch1_1 in range(now(), -20y)")
res = q.run()

for col in res.tables["ts_batch1_1"]:
    print(col.name, ": ", col.data)

$timestamp :  ['2019-03-19T00:00:00.000000000']
min(close) :  [16.47828926]


In [27]:
q = c.query("select arithmetic_mean(close) from ts_batch1_1 in range(2019, +1y) group by day")
res = q.run()
for col in res.tables["ts_batch1_1"]:
    print(col.name, ": ", col.data)

$timestamp :  ['2019-03-14T00:00:00.000000000' '2019-03-15T00:00:00.000000000'
 '2019-03-16T00:00:00.000000000' '2019-03-17T00:00:00.000000000'
 '2019-03-18T00:00:00.000000000' '2019-03-19T00:00:00.000000000'
 '2019-03-20T00:00:00.000000000' '2019-03-21T00:00:00.000000000'
 '2019-03-22T00:00:00.000000000' '2019-03-23T00:00:00.000000000']
arithmetic_mean(close) :  [75.68424133 52.2474901  92.13038454 59.42907505 57.19005246 16.47828926
 50.99041598 75.97125142 97.64092279 45.21851715]


<b>You can use an iterator on the table to access access data without using a query</b>

In [28]:
for row in table.reader(dict=True):
    print(row['close'])


75.68424133378036
52.24749009852676
92.13038454037721
59.42907504604708
57.190052458697984
16.478289263003965
50.99041598258343
75.97125141839608
97.64092278815458
45.21851715245746


In [29]:
assert table.column_type_by_id('volume')== quasardb.quasardb.ColumnType.Int64
assert table.column_type_by_id('close')== quasardb.quasardb.ColumnType.Double

In [30]:
#tearing down data for this sample
table = c.ts("ts_batch1_1")
table.remove()